In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import  xml.dom.minidom
import os
import re
import linecache
from xml.parsers.expat import ExpatError
import torch
!pip install openprompt
import numpy as np
import time
import datetime
from openprompt.data_utils import InputExample
import random
from itertools import zip_longest
from random import shuffle
from openprompt.plms import load_plm
from openprompt import PromptDataLoader
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from openprompt.prompts import ManualTemplate, MixedTemplate, SoftTemplate, PrefixTuningTemplate
from openprompt.prompts import ManualVerbalizer, SoftVerbalizer
from openprompt import PromptForClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from openprompt import PromptForClassification
import pandas as pd
from openprompt.plms import T5TokenizerWrapper


In [ ]:
import os
import matplotlib.pyplot as plt

folder_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/2012-07-15.original-annotation.release"

word_counts = {}

# Iterate through all files in the folder
for file in os.listdir(folder_path):
    if file.endswith('.txt'):
        file_path = os.path.join(folder_path, file)
        
        # Read the file line by line
        with open(file_path, 'r') as f:
            for line in f:
                words = line.split()
                word_count = len(words)
                
                if word_count in word_counts:
                    word_counts[word_count] += 1
                else:
                    word_counts[word_count] = 1

# Sort the word counts by the number of words in a sentence
sorted_word_counts = sorted(word_counts.items())

# Separate the sorted data into two lists for plotting
word_count_values, sentence_counts = zip(*sorted_word_counts)

# Create a bar chart
plt.bar(word_count_values, sentence_counts)
plt.xlabel('Number of words in a sentence')
plt.ylabel('Number of sentences')
plt.title('Word count distribution in sentences')
plt.show()


In [ ]:

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


#generate gold labels by rule based method
def gold_labeling(path, window_size_before_target_sentence, window_size_after_target_sentence):
    files = os.listdir(path)
    files.sort()

    dataset = {}
    treatment_list = []
    tlink_list = []
    admission = ""
    discharge = ""
    for file in files:

        #extract treatment entities and time information
        if file.endswith(".extent"):

            treatment_list = []

            f = open(path+"/"+file)
            for line in f:
                s = re.split(r'[||""\'\']',line)
                if s[0] == "EVENT=":
                    if s[5] == "TREATMENT":
                        treatment_list.append(s[1])
                elif s[0] == "SECTIME=":
                    if s[5] == "ADMISSION":
                        admission = s[1]
                    elif s[5] == "DISCHARGE":
                        discharge = s[1]

        #extract temporal link information
        elif file.endswith(".tlink"):
            tlink_list = []

            t = open(path+"/"+file)
            for line in t:
                tlink = re.split(r'[||""\'\']',line)
                for i in treatment_list:
                    if tlink[1] == i:
                        begin = [int(x) for x in tlink[2].split()[0].split(":")]
                        end = [int(y) for y in tlink[2].split()[1].split(":")]
                        txt_path = (path+"/"+file).replace(".tlink", ".txt")
                        line_number = begin[0]-1
                        txt_content = [x.replace("\n", " ") for x in linecache.getlines(txt_path)]
                        text_a = txt_content[:4] +[". Dorctor notes: "]

                        #context window
                        #One sentence before: One sentence after
                        #[line_number-1 : line_number+2]
                        target_sentent = txt_content[line_number - window_size_before_target_sentence : (line_number+1)+window_size_after_target_sentence]
                        text_a += target_sentent
                        if tlink[5] == admission and tlink[9] == "BEFORE":
                            tlink_list.append([i, "OFF", begin, end, "".join(text_a)])
                        elif tlink[5] == admission and tlink[9] == "AFTER":
                            tlink_list.append([i, "ON", begin, end, "".join(text_a)])
                        elif tlink[5] == discharge and tlink[9] == "BEFORE":
                            tlink_list.append([i, "ON", begin, end, "".join(text_a)])
                        else:
                            pass

            dataset[file] = tlink_list
            
    return dataset


def get_train_set(train_path, number_ON_label, number_OFF_label, window_size_before_target_sentence, window_size_after_target_sentence):
    #train_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/2012-07-15.original-annotation.release"

    train_data = gold_labeling(train_path, window_size_before_target_sentence, window_size_after_target_sentence)
    #print(train_data)
    label_ON_dataset = []
    label_OFF_dataset = []

    train_dataset_prompt = []
    train_dataset_fine = []
    count = 0
    train_ON_count = 0
    train_OFF_count = 0
    total_number_ON = 0
    total_number_OFF = 0
    for file in train_data:

        for gold_label in train_data[file]:
            if gold_label[1] == "ON":
                label_ON_dataset.append([file]+gold_label)
                total_number_ON+=1
            else:
                label_OFF_dataset.append([file]+gold_label)
                total_number_OFF+=1
    
    #few-shot training set
    if number_ON_label<=total_number_ON :
        train_label_ON_dataset = random.sample(label_ON_dataset, number_ON_label)
    else :
        train_label_ON_dataset = label_ON_dataset
    if number_OFF_label<=total_number_OFF:
        train_label_OFF_dataset = random.sample(label_OFF_dataset, number_OFF_label)
    else :
        train_label_OFF_dataset = label_OFF_dataset
    
    #randomly merge label ON and OFF dataset
    print("train_label_ON_dataset",len(train_label_ON_dataset))
    print("train_label_OFF_dataset",len(train_label_OFF_dataset))
    temp_list = list(zip_longest(train_label_ON_dataset, train_label_OFF_dataset, fillvalue=None))
    shuffle(temp_list)

    merged_label_dataset = []
    for item in temp_list:
        if item[0] is not None:
            merged_label_dataset.append(item[0])
        if item[1] is not None:
            merged_label_dataset.append(item[1])

    print(len(merged_label_dataset))
    for gold_label in merged_label_dataset:
        if gold_label[2] == "ON":
            train_label = 1
            train_ON_count += 1
        else:
            train_label = 0
            train_OFF_count += 1

        #generate training data for prompt-based leanring
        input_example = InputExample(text_a = gold_label[5], text_b = gold_label[1], label = train_label, guid = count, meta = gold_label[0])
        count += 1
        #generate training data for fine-tuning
        train_dataset_prompt.append(input_example)
        train_dataset_fine.append([gold_label[5], gold_label[1], train_label, count, gold_label[0]])

    print(input_example)
    print("number of ON train",train_ON_count)
    print("number of OFF train",train_OFF_count)
    print("total number of training set", count)

    
    return (train_dataset_prompt, train_dataset_fine)


def get_test_set (test_path, window_size_before_target_sentence, window_size_after_target_sentence):

    test_ON_count = 0
    test_OFF_count = 0
    #test_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/ground_truth/merged_i2b2"
    test_data = gold_labeling(test_path, window_size_before_target_sentence, window_size_after_target_sentence)
    #print(test_data)
    test_dataset_prompt = []
    test_dataset_fine = []
    count = 0
    for file in test_data:
        
        #creat training labels
        for gold_label in test_data[file]:
            if gold_label[1] == "ON":
                test_label = 1
                test_ON_count += 1
            else:
                test_label = 0
                test_OFF_count += 1
            
            input_example = InputExample(text_a = gold_label[4], text_b = gold_label[0], label = test_label, guid = count, meta = file)
            count += 1
            test_dataset_prompt.append(input_example)
            test_dataset_fine.append([gold_label[4], gold_label[0], test_label, count, file])

    print("number of ON test",test_ON_count)

    print("number of OFF test",test_OFF_count)

    return (test_dataset_prompt, test_dataset_fine)

In [ ]:
 #different pre-trained language model
def select_plm (option):
    if option == 0:
        return load_plm("bert", "bert-base-uncased")
    elif option == 1:
        return load_plm("roberta", "roberta-base")
    elif option == 2:
        return load_plm("t5", "t5-base")
    elif option == 3:
        return load_plm("gpt2", "gpt2")
    else :
        print("there is no option "+ option +"in PLM selection")
        return None

    #return format: plm, tokenizer, model_config, WrapperClass


#different template and verbalizer
#manual, soft, mixed, prefix
def select_template_and_verbalizer (option, tokenizer, plm):
    if option == 0:
        return (
        ManualTemplate(tokenizer=tokenizer, text='{"placeholder":"text_a"} Question: {"placeholder":"text_b"} were used between admission and discharge time. Is it correct? {"mask"}.'),
        ManualVerbalizer(tokenizer, num_classes=2, label_words=[["yes"], ["no"]])
        )
    elif option == 1:
        return (
        ManualTemplate(tokenizer=tokenizer, text='{"placeholder":"text_a"} In this paragraph of the note, {"placeholder":"text_b"} {"mask"} used between admission and discharge time.'),
        ManualVerbalizer(tokenizer, num_classes=2, label_words=[["is"], ["not"]])
        )
    elif option == 2:
        return (
        ManualTemplate(tokenizer=tokenizer, text='{"placeholder":"text_a"} In this paragraph of the note, {"placeholder":"text_b"} {"mask"} used between admission and discharge time.'),
        ManualVerbalizer(
        tokenizer = tokenizer,
        num_classes = 2, 
        classes = ["ON", "OFF"],
        label_words = {
            "ON": ["definitely"],
            "OFF": ["possibly","probably","difinitely not"],
            })
        )
    elif option == 3:
        return (
        ManualTemplate(tokenizer=tokenizer, text='{"placeholder":"text_a"} Question: {"placeholder":"text_b"} were used between admission and discharge time. Is it correct? {"mask"}.'),
        SoftVerbalizer(tokenizer, plm, num_classes=2)
        )
    elif option == 4:
        return (
        #only for seq2seq, how about altoregressive model.
        PrefixTuningTemplate(model=plm, tokenizer=tokenizer, text='{"placeholder":"text_a"} Question: {"placeholder":"text_b"} were used between admission and discharge time. Is it correct? {"mask"}'),
        SoftVerbalizer(tokenizer, plm, num_classes=2)
        )

    elif option == 5:
        return (
        MixedTemplate(model=plm, tokenizer=tokenizer, text='{"placeholder":"text_a"} {"soft": "Question:"} {"placeholder":"text_b"} {"soft"} {"soft"} {"soft"} {"soft"} {"soft"} Is it correct? {"mask"}.'),
        ManualVerbalizer(tokenizer, num_classes=2, label_words=[["yes"], ["no"]])
        )
    elif option == 6:
        return (
        MixedTemplate(model=plm, tokenizer=tokenizer, text='{"placeholder":"text_a"} In this paragraph of the note, {"placeholder":"text_b"} {"soft"} {"mask"} {"soft"} {"soft"} {"soft"} {"soft"} {"soft"} {"soft"}'),
        SoftVerbalizer(tokenizer, plm, num_classes=2)
        )
    else:
        print("we do not have option "+option)


collect data

In [ ]:
#set training data and testing data
#able to use few-shot method
train_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/2012-07-15.original-annotation.release"
number_ON_label = 5
number_OFF_label = 5
window_size_before_target_sentence = 3
window_size_after_target_sentence = 2
train_dataset_prompt, train_dataset_tuning = get_train_set(train_path, number_ON_label, number_OFF_label, window_size_before_target_sentence, window_size_after_target_sentence)


test_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/ground_truth/merged_i2b2"
test_dataset_prompt ,test_dataset_tuning = get_test_set(test_path, window_size_before_target_sentence, window_size_after_target_sentence)


In [ ]:
# display input informations
train_dataframe = pd.DataFrame(train_dataset_tuning , columns=["discharge note", "treatment entity", "label(1-ON,0-OFF)", "guid", "document name"])
train_dataframe = train_dataframe.reindex(columns=["document name","discharge note","treatment entity", "label(1-ON,0-OFF)"])
train_dataframe


In [ ]:
test_dataframe = pd.DataFrame(test_dataset_tuning , columns=["doctor note", "treatment entity", "label", "guid", "document name"])
test_dataframe = test_dataframe.reindex(columns=["document name","doctor note","treatment entity", "label"])
test_dataframe

 Prompt based learning

In [ ]:
plm, tokenizer, model_config, WrapperClass = select_plm(0)

mytemplate, myverbalizer = select_template_and_verbalizer(0, tokenizer, plm)

#how dose the template look like
wrapped_example = mytemplate.wrap_one_example(train_dataset_prompt[0])
print(wrapped_example)

text_list = []

for item in wrapped_example[0]:
    text_list.append(item['text'])

df = pd.DataFrame({'text': text_list})
df

# wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
# print(tokenized_example)

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
#prompt dataloader

train_dataloader_prompt = PromptDataLoader(dataset=train_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method='head')
#truncate_method=`head`, `tail`, `balanced`
#max_seq_length (int, optional) – The max sequence length of the input ids. It’s used to truncate sentences.

validation_dataloader_prompt = PromptDataLoader(dataset=test_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")


#training
use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

for epoch in range(5):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader_prompt):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step %100 ==1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)


#evaluation
allpreds = []
alllabels = []
allids = []
for step, inputs in enumerate(validation_dataloader_prompt):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    guids = inputs['guid']
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    allids.extend(guids.cpu().tolist())

accuracy = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)



TP = 0
TN = 0
FP = 0
FN = 0
errorIDs = []
count = 0
for i,j,id in zip(allpreds, alllabels, allids):
    count += 1
    if j == 1:
        if i == 1:
            TP += 1
        else:
            FN += 1
            errorIDs.append(id)
    else:
        if i == 1:
            FP += 1
            errorIDs.append(id)
        else:
            TN += 1

print(TP, FN, FP, TN)
#confusion matrix
precision = TP/(TP+FP)
recall = TP/(TP+FN)
F1 = 2*(precision*recall)/(precision + recall)


print("overall accuracy:",accuracy)
print("ON accuracy:     ",(TP)/(TP+FN))
print("OFF accuracy:    ",(FP)/(FP+TN))
print("Precision:       ",precision)
print("Recall:          ",recall)
print("F1:              ",F1)


In [ ]:
#error analysis
error_inputs = [x for x in test_dataset_prompt if x.guid in errorIDs]
# print(error_inputs)

fine-tuning

In [ ]:
#load tokenizer for fine-tuning plm, and set the maximum sequence length 
tokenizer_tuning = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

max_len = 0

for dataset in train_dataset_tuning:
    input = dataset[0]+'Treament: '+dataset[1]
    input_ids = tokenizer_tuning.encode(input, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print(input)        
print(max_len)

这里的‘ ：’格式有点奇怪，句号都是在空格后

In [ ]:
input_ids = []
attention_masks = []
labels = [i[2] for i in train_dataset_tuning]
for dataset in train_dataset_tuning:
    input = dataset[0]+'Treatment: '+dataset[1]
    encoded_dict = tokenizer_tuning.encode_plus(
                    input,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = 276,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
                )
    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)



####set the training and validation set####

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


####dataloader####

batch_size = 4

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )





model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 5

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

device = torch.device("cuda")

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')


    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        model.zero_grad()        


        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)
        loss = result.loss
        logits = result.logits

        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        

        with torch.no_grad():        

            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)
        loss = result.loss
        logits = result.logits
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



#Evaluation


# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
test_labels = [i[2] for i in test_dataset_tuning]

# For every sentence...
for dataset in test_dataset_tuning:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    input = dataset[0]+ 'Treatment: '+dataset[1]

    encoded_dict = tokenizer_tuning.encode_plus(
                        input,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 276,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(test_labels)

batch_size = 64

prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels, pred_labels = [], [], []

# Predict 
for batch in prediction_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
      result = model(b_input_ids, 
                     token_type_ids=None, 
                     attention_mask=b_input_mask,
                     return_dict=True)

  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels

  #predictions.append(logits)
  pred_labels.extend(np.argmax(logits, axis=1).flatten())
  
  true_labels.extend(label_ids)


accuracy = sum([int(i==j) for i,j in zip(pred_labels, true_labels)])/len(pred_labels)

TP = 0
TN = 0
FP = 0
FN = 0
count = 0
for i,j, in zip(pred_labels, true_labels):
    count += 1
    if j == 1:
        if i == 1:
            TP += 1
        else:
            FN += 1
    else:
        if i == 1:
            FP += 1
        else:
            TN += 1

#
precision = TP/(TP+FP)
recall = TP/(TP+FN)
F1 = 2*(precision*recall)/(precision + recall)


print("overall accuracy:",accuracy)
print("ON accuracy:     ",(TP)/(TP+FN))
print("OFF accuracy:    ",(TN)/(FP+TN))
print("Precision:       ",precision)
print("Recall:          ",recall)
print("F1:              ",F1)



graph and compare


In [ ]:
#compare models and paradigm
number_of_training = [5, 10, 20 ,40]
number_of_template = [1, 6]
accuracy_list_prompt = []
accuracy_list_tuning = []
f1_list_prompt = []
f1_list_tuning = []

for i in number_of_training:
    train_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/2012-07-15.original-annotation.release"
    number_ON_label = i
    number_OFF_label = i
    window_size_before_target_sentence = 2
    window_size_after_target_sentence = 3
    train_dataset_prompt, train_dataset_tuning = get_train_set(train_path, number_ON_label, number_OFF_label, window_size_before_target_sentence, window_size_after_target_sentence)

    test_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/ground_truth/merged_i2b2"
    test_dataset_prompt ,test_dataset_tuning = get_test_set(test_path, window_size_before_target_sentence, window_size_after_target_sentence)


    input_ids = []
    attention_masks = []
    labels = [i[2] for i in train_dataset_tuning]
    for dataset in train_dataset_tuning:
        input = dataset[0]+'Treatment: '+dataset[1]
        encoded_dict = tokenizer_tuning.encode_plus(
                        input,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 276,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                    )
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    # Combine the training inputs into a TensorDataset.
    dataset = TensorDataset(input_ids, attention_masks, labels)

    # Create a 90-10 train-validation split.

    # Calculate the number of samples to include in each set.
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size

    # Divide the dataset by randomly selecting samples.
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    print('{:>5,} training samples'.format(train_size))
    print('{:>5,} validation samples'.format(val_size))


    batch_size = 4

    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )

    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 2, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )
    model.cuda()

    optimizer = AdamW(model.parameters(),
                    lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                    )

    # Number of training epochs. The BERT authors recommend between 2 and 4. 
    # We chose to run for 4, but we'll see later that this may be over-fitting the
    # training data.
    epochs = 5

    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)

    device = torch.device("cuda")

    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):
        
        # ========================================
        #               Training
        # ========================================
        
        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
                
                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        

            result = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels,
                        return_dict=True)
            loss = result.loss
            logits = result.logits

            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            
        
        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
            
        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            
            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                result = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels,
                            return_dict=True)
            loss = result.loss
            logits = result.logits
            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)
            

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        
        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)
        
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []
    test_labels = [i[2] for i in test_dataset_tuning]

    # For every sentence...
    for dataset in test_dataset_tuning:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        input = dataset[0]+ 'Treatment: '+dataset[1]

        encoded_dict = tokenizer_tuning.encode_plus(
                            input,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 276,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(test_labels)

    batch_size = 4

    prediction_data = TensorDataset(input_ids, attention_masks, labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


    # Prediction on test set

    print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

    # Put model in evaluation mode
    model.eval()

    # Tracking variables 
    predictions , true_labels, pred_labels = [], [], []

    # Predict 
    for batch in prediction_dataloader:
    # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and 
        # speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            result = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            return_dict=True)

        logits = result.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Store predictions and true labels

        #predictions.append(logits)
        pred_labels.extend(np.argmax(logits, axis=1).flatten())

        true_labels.extend(label_ids)


    accuracy = sum([int(i==j) for i,j in zip(pred_labels, true_labels)])/len(pred_labels)

    TP = 0
    TN = 0
    FP = 0
    FN = 0
    count = 0
    for i,j, in zip(pred_labels, true_labels):
        count += 1
        if j == 1:
            if i == 1:
                TP += 1
            else:
                FN += 1
        else:
            if i == 1:
                FP += 1
            else:
                TN += 1

    precision = TP/(TP+FP)#预测为正类中有多少是label正的
    recall = TP/(TP+FN)#label正的中有多少被预测出来
    F1 = 2*(precision*recall)/(precision + recall)


    print("overall accuracy:",accuracy)
    print("F1:              ",F1)
    f1_list_tuning.append(F1)
    accuracy_list_tuning.append(accuracy)




    temp_f1_prompt = []
    temp_acc_prompt = []
    for j in number_of_template:


        plm, tokenizer, model_config, WrapperClass = select_plm(2)

        mytemplate, myverbalizer = select_template_and_verbalizer(j, tokenizer, plm)
        train_dataloader_prompt = PromptDataLoader(dataset=train_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
            batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        validation_dataloader_prompt = PromptDataLoader(dataset=test_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
            batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        use_cuda = True
        prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
        if use_cuda:
            prompt_model=  prompt_model.cuda()


        from transformers import  AdamW, get_linear_schedule_with_warmup
        loss_func = torch.nn.CrossEntropyLoss()
        no_decay = ['bias', 'LayerNorm.weight']
        # it's always good practice to set no decay to biase and LayerNorm parameters
        optimizer_grouped_parameters = [
            {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

        for epoch in range(5):
            tot_loss = 0
            for step, inputs in enumerate(train_dataloader_prompt):
                if use_cuda:
                    inputs = inputs.cuda()
                logits = prompt_model(inputs)
                labels = inputs['label']
                loss = loss_func(logits, labels)
                loss.backward()
                tot_loss += loss.item()
                optimizer.step()
                optimizer.zero_grad()
                #if step %100 ==1:
                    #print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

        # Evaluate

        allpreds = []
        alllabels = []
        allids = []
        for step, inputs in enumerate(validation_dataloader_prompt):
            if use_cuda:
                inputs = inputs.cuda()
            logits = prompt_model(inputs)
            guids = inputs['guid']
            labels = inputs['label']
            alllabels.extend(labels.cpu().tolist())
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
            allids.extend(guids.cpu().tolist())

        acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
        temp_acc_prompt.append(acc)
        print(acc)

        #confusion matrix
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        errorIDs = []
        count = 0
        for i,j,id in zip(allpreds, alllabels, allids):
            count += 1
            if j == 1:
                if i == 1:
                    TP += 1
                else:
                    FN += 1
                    errorIDs.append(id)
            else:
                if i == 1:
                    FP += 1
                    errorIDs.append(id)
                else:
                    TN += 1


        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        F1 = 2*(precision*recall)/(precision + recall)
        # print("overall accuracy:",accuracy)
        # print("ON accuracy:     ",(TP)/(TP+FN))
        # print("OFF accuracy:    ",(FP)/(FP+TN))
        # print("Precision:       ",precision)
        # print("Recall:          ",recall)
        print("F1:",F1)
        temp_f1_prompt.append(F1)
    f1_list_prompt.append(temp_f1_prompt)
    accuracy_list_prompt.append(temp_acc_prompt)

        


In [ ]:
print(accuracy_list_prompt)
template_a_acc = [i[0] for i in accuracy_list_prompt]
template_b_acc = [i[1] for i in accuracy_list_prompt]

template_a_f1 = [i[0] for i in f1_list_prompt]
template_b_f1 = [i[1] for i in f1_list_prompt]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(number_of_training, accuracy_list_tuning, label='fine-tuning')
plt.plot(number_of_training, template_a_acc, label='template 1')
plt.plot(number_of_training, template_b_acc, label='template 6') 
plt.title("few shot learning with accuracy")
plt.xlabel("number of data in each label")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.plot(number_of_training, f1_list_tuning, label='fine-tuning')
plt.plot(number_of_training, template_a_f1, label='template 1')
plt.plot(number_of_training, template_b_f1, label='template 6')
plt.title("few shot learning with F1 score")
plt.xlabel("number of data in each label")
plt.ylabel("F1 score")
plt.legend()
plt.show()

In [ ]:
#compare models and paradigm
number_of_training = [5, 10, 20 ,40, 80, 160, 320, 762]
number_of_plm = [0,1]
accuracy_list_prompt = []
accuracy_list_tuning = []
f1_list_prompt = []
f1_list_tuning = []

for i in number_of_training:
    train_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/2012-07-15.original-annotation.release"
    number_ON_label = i
    number_OFF_label = i
    window_size_before_target_sentence = 1
    window_size_after_target_sentence = 1
    train_dataset_prompt, train_dataset_tuning = get_train_set(train_path, number_ON_label, number_OFF_label, window_size_before_target_sentence, window_size_after_target_sentence)

    test_path = "/content/drive/MyDrive/3rd_year_project/2012_Temporal_Relations_Challenge/ground_truth/merged_i2b2"
    test_dataset_prompt ,test_dataset_tuning = get_test_set(test_path, window_size_before_target_sentence, window_size_after_target_sentence)



    temp_f1_prompt = []
    temp_acc_prompt = []
    for j in number_of_plm:


        plm, tokenizer, model_config, WrapperClass = select_plm(j)

        mytemplate, myverbalizer = select_template_and_verbalizer(0, tokenizer, plm)
        train_dataloader_prompt = PromptDataLoader(dataset=train_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
            batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        validation_dataloader_prompt = PromptDataLoader(dataset=test_dataset_prompt, template=mytemplate, tokenizer=tokenizer,
            tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
            batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
            truncate_method="head")

        use_cuda = True
        prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
        if use_cuda:
            prompt_model=  prompt_model.cuda()


        from transformers import  AdamW, get_linear_schedule_with_warmup
        loss_func = torch.nn.CrossEntropyLoss()
        no_decay = ['bias', 'LayerNorm.weight']
        # it's always good practice to set no decay to biase and LayerNorm parameters
        optimizer_grouped_parameters = [
            {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

        optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)

        for epoch in range(5):
            tot_loss = 0
            for step, inputs in enumerate(train_dataloader_prompt):
                if use_cuda:
                    inputs = inputs.cuda()
                logits = prompt_model(inputs)
                labels = inputs['label']
                loss = loss_func(logits, labels)
                loss.backward()
                tot_loss += loss.item()
                optimizer.step()
                optimizer.zero_grad()
                #if step %100 ==1:
                    #print("Epoch {}, average loss: {}".format(epoch, tot_loss/(step+1)), flush=True)

        # Evaluate

        allpreds = []
        alllabels = []
        allids = []
        for step, inputs in enumerate(validation_dataloader_prompt):
            if use_cuda:
                inputs = inputs.cuda()
            logits = prompt_model(inputs)
            guids = inputs['guid']
            labels = inputs['label']
            alllabels.extend(labels.cpu().tolist())
            allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
            allids.extend(guids.cpu().tolist())

        acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
        temp_acc_prompt.append(acc)
        print(acc)

        #confusion matrix
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        errorIDs = []
        count = 0
        for i,j,id in zip(allpreds, alllabels, allids):
            count += 1
            if j == 1:
                if i == 1:
                    TP += 1
                else:
                    FN += 1
                    errorIDs.append(id)
            else:
                if i == 1:
                    FP += 1
                    errorIDs.append(id)
                else:
                    TN += 1

        
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        F1 = 2*(precision*recall)/(precision + recall)
        print("overall accuracy:",acc)
        print("ON accuracy:     ",(TP)/(TP+FN))
        print("OFF accuracy:    ",(TN)/(FP+TN))
        print("Precision:       ",precision)
        print("Recall:          ",recall)
        print("F1:",F1)
        temp_f1_prompt.append(F1)
    f1_list_prompt.append(temp_f1_prompt)
    accuracy_list_prompt.append(temp_acc_prompt)

        


In [ ]:
import matplotlib.pyplot as plt

print(accuracy_list_prompt)
print(f1_list_prompt)
template_a_acc = [i[0] for i in accuracy_list_prompt]
template_b_acc = [i[1] for i in accuracy_list_prompt]
template_a_f1 = [i[0] for i in f1_list_prompt]
template_b_f1 = [i[1] for i in f1_list_prompt]

plt.rcParams["figure.figsize"] = (12,6)
plt.plot(number_of_training[:-1], template_a_acc, label='BERT accuracy')
plt.plot(number_of_training[:-1], template_b_acc, label='RoBERT accuracy') 
plt.title("few shot learning with accuracy")
plt.xlabel("number of data in each label")
plt.ylabel("accuracy")
plt.legend()
plt.show()

plt.rcParams["figure.figsize"] = (12,6)
plt.plot(number_of_training[:-1], template_a_f1, label='BERT F1')
plt.plot(number_of_training[:-1], template_b_f1, label='RoBERT F1') 
plt.title("few shot learning with F1")
plt.xlabel("number of data in each label")
plt.ylabel("F1")
plt.legend()
plt.show()